# BT01: Thu thập dữ liệu từ web

TODO: Võ Phương Hòa - 1412192

---

## 1. Tổng thể

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm. Trong quá trình làm, thường xuyên `Ctrl+S` để lưu lại bài làm.

Nên nhớ mục tiêu chính ở đây là *học, học một cách chân thật*. Bạn có thể thảo luận ý tưởng với bạn khác cũng như là tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn, dựa trên sự hiểu của bạn*. Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.

**Cách nộp bài**

Trước khi nộp bài, bạn chọn `Kernel` -> `Restart & Run All` (restart python và chạy tất cả các cell), rồi kiểm tra xem có bị lỗi gì không.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt file `BT01-ThuThapDuLieuWeb.ipynb`, rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

**Nội dung bài tập**

Trong bài này, bạn sẽ ôn lại ví dụ "Nhà Khoa Học Dữ Liệu cần biết ngôn ngữ lập trình nào?" mà mình đã hướng dẫn ở trên lớp. Cụ thể là bạn sẽ:
- Đọc lại phần code mà mình đã hướng dẫn trên lớp.
- Bổ sung phần trực quan hóa (visualize) kết quả.

## 2. Import

In [ ]:
%matplotlib inline
import requests # To read html text
from bs4 import BeautifulSoup # To parse html text
import re # Regular expressions
# You can also import other thing ...
import matplotlib.pyplot as plt

## 3. Hàm rút trích tập từ từ một trang web

In [ ]:
def extract_word_set(url):
    '''
    Extracts word set from a web.
    
    Parameters
    ----------
    url : string
        The url of the web.
    
    Returns
    -------
    word_set : set
        The set of extracted words.
    '''
    # Read html text
    try:
        html_text = requests.get(url).text
    except:
        print 'Cannot read ' + url
        return set([])
    
    # Parse html text
    tree = BeautifulSoup(html_text)
    
    # Remove invisible tags
    for invisible_tag in tree.find_all(['script', 'style']): 
        invisible_tag.extract()
    
    # Get texts from visible tags, concatenate them using spaces, and then lower all characters
    text = tree.get_text(" ").lower()
    
    # Substitute spaces for unwanted things 
    filtered_text = re.sub("[^a-z+#]", " ", text)
    
    # Split filtered text into words, and remove duplicate words
    word_set = set(filtered_text.split())

    return word_set

## 4. Chương trình chính

In [ ]:
# Read html text and parse it
url = 'https://www.indeed.com/jobs?q=data+scientist&l='
html_text = requests.get(url).text
tree = BeautifulSoup(html_text)

# Extract num jobs
num_jobs = int(tree.find(id='searchCount').string.split()[-1].replace(",", ""))
print 'num_jobs =', num_jobs

# Compute num pages
num_pages = (num_jobs - 1) / 10 + 1 # 10 jobs / page
print 'num_pages =', num_pages
num_pages = 50 # It's enough to wait
print 'Here we collect first %d pages' %(num_pages)

# Init language counts
language_counts = {'python': 0, 'r': 0, 'matlab': 0, 'c++': 0, 'c#': 0, 'java': 0}

# With each page ...
for page_idx in range(num_pages):
    print '* Page', page_idx
    
    # Read html text and parse it
    url = 'https://www.indeed.com/jobs?q=data+scientist&start=' + str(page_idx * 10) + '&pp=ABQAAAFakt3ssAAAAAEEW-h_AQEJCAMPSIP4V9-fK7_-Ic_UdBI_lj6hG672oxZDRHZXK26t_QYh9GTXmeLByGiiv7VhGVV8'
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    
    # Find job tags
    job_tag_set = tree.find_all(class_='result')
    
    # With each job tags ...
    for job_tag in job_tag_set:
        # Extract job url
        job_url = 'https://www.indeed.com' + job_tag.find('a')['href']
        
        # Extract word set from job url
        word_set = extract_word_set(job_url)
        
        # Update language counts
        for language in language_counts.keys():
            if language in word_set:
                language_counts[language] += 1

print                
print '='*70                
print language_counts

Dùng bar chart để visualize kết quả (trục hoành là ngôn ngữ, trục tung là giá trị đếm; để dễ nhìn, bạn sort các ngôn ngữ theo giá trị đếm giảm dần).

In [ ]:
# TODO
li_keys = []
li_values= []
sorted_keys = []
sorted_values = []

# take keys and values from dict to list
for key in language_counts.keys():
    li_keys.append(key)
for value in language_counts.values():
    li_values.append(value)  

# sort values from max to min
for i in range(len(li_keys)):
    value = max(li_values)
    ind = li_values.index(value)
    sorted_keys.append(li_keys[ind])
    sorted_values.append(li_values[ind])
    del li_values[ind]
    del li_keys[ind]
    
# draw values by bar chart
plt.bar(range(len(sorted_keys)), sorted_values, width = 0.4, tick_label = sorted_keys, align = 'center')
plt.axis([-1, len(sorted_keys), 0, int(max(sorted_values) * 1.2)])

plt.show()

Có nên học python :-)